# Aprendizaje automátco relacional

# 1 Lectura de datos y creación del DataFrame con las métricas

In [2]:
import pandas as pd
import networkx as nx
import numpy as np
np.random.seed(357823)
import random
random.seed(357823)

En primer lugar, realizamos la lectura de los vértices y aristas y creamos un grafo con estas.

In [3]:
edges=pd.read_csv('edges.csv')
vertices=pd.read_csv('nodes.csv')

In [4]:
edges.head()

,source,target
0,Baron Zemo,Blackout (Marcus Daniels)
1,Baron Zemo,Vermin (comics)
2,N'astirh,S'ym
3,N'astirh,Belasco (Marvel Comics)
4,N'astirh,Madelyne Pryor


In [5]:
vertices.head()

,group,id,size
0,1,Baron Zemo,2
1,1,N'astirh,3
2,0,Silver Sable,1
3,0,Hope Summers (comics),1
4,0,Magik (Illyana Rasputin),1


In [6]:
G=nx.from_pandas_edgelist(edges,source='source',target='target')

Obtenemos las métricas relacionales para cada nodo. Hemos decidido filtrar los vértices según el tamaño de su componente conexa, ya que quedan algunos vértices aislados en componentes conexas muy pequeñas, donde las métricas relacionales no tienen sentido. Tras experimentación el umbral que se ha decido escoger es de 10.

In [7]:
from collections import Counter
degree=nx.degree(G)
triangles=nx.triangles(G)
degree_centrality= nx.degree_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)
eigenvector_centrality = nx.eigenvector_centrality(G)
clustering = nx.clustering(G)
tam_comp = {}
for i, component in enumerate(nx.connected_components(G)):
    for node in component:
        tam_comp[node] = len(component)
vertices = vertices[vertices['id'].map(lambda v: tam_comp[v] >= 10)]

Convertimos las métricas obtenidas a lista para usarlas para crear el dataframe de los datos, utilizamos los vértices leídos del dataset al iterar para garantizar que todas tengan el mismo orden y que no se mezclen métricas de diferentes vértices.

In [8]:
degree_dict= {d[0]:d[1] for d in degree}
degreeL = [degree[v["id"]]for _,v in vertices.iterrows()]
trianglesL=[triangles[v['id']] for _,v in vertices.iterrows()]
degree_centralityL = [degree_centrality[v['id']] for _,v in vertices.iterrows()]
closeness_centralityL = [closeness_centrality[v['id']] for _,v in vertices.iterrows()]
betweenness_centralityL = [betweenness_centrality[v['id']] for _,v in vertices.iterrows()]
eigenvector_centralityL =[eigenvector_centrality[v['id']] for _,v in vertices.iterrows()]
clusteringL = [clustering[v['id']] for _,v in vertices.iterrows()]
# component_idL= [component_id[v['id']] for _,v in vertices.iterrows()]

Crear el dataframe con los datos.

In [9]:
df = pd.DataFrame({'triangle':trianglesL, "degree_centrality":degree_centralityL,
                  'closeness_centrality':closeness_centralityL, 'betweenness_centrality':betweenness_centralityL,
                    'eigenvector_centrality':eigenvector_centralityL,'clustering':clusteringL,'group':vertices['group']})



In [10]:
df.head()

,triangle,degree_centrality,closeness_centrality,betweenness_centrality,eigenvector_centrality,clustering,group
3,0,0.002865,0.065194,0.000000,1.361871e-03,0.000000,0
6,0,0.002865,0.009551,0.000000,1.666953e-17,0.000000,1
9,0,0.017192,0.110388,0.176275,1.224383e-01,0.000000,1
14,0,0.002865,0.052361,0.000000,3.509777e-05,0.000000,2
16,1,0.008596,0.070706,0.011593,6.506930e-04,0.333333,1


In [11]:
atributos=df.iloc[:,0:-1]
objetivo=df.iloc[:,-1]

# 2 Entrenamiento y evalución de los distintos modelos

LLevamos a cabo la división del conjunto de ejemplos de ejemplos en entrenamiento y prueba. Escoegmos el 80% como conjunto de entrenamiento y el 20% como conjunto de prueba. Se usa el parámetro stratify para garantizar que la proporción de las clases sea la misma en el conjunto de entrenamiento y en el de prueba.

In [12]:
from sklearn.model_selection import train_test_split
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos, objetivo,
        # Tamaño del conjunto de prueba (20 % en este caso)
        test_size=.2,
        # Estratificación según la distribución de clases en el atributo objetivo
        stratify=objetivo,random_state=357823)

Se normalizan los datos para garantizar que todas las propiedades contribuyan de igual medida y evitar desbordamientos numéricos.

In [13]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

normalizador = ColumnTransformer(
    [
        ('normalizador', MinMaxScaler(), list(range(0, 6)))
    ]
)

Selección de mejores hiperparámetros para el Árbol de Decisión CART utilizando GridSearch:

In [14]:
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
rejilla_de_hiperparámetros = {
    # Máxima profundidad del árbol: 3, 4, 5, 6, 7, 8, 9, 10
    'arbol__max_depth': range(3, 11),
    # Mínimo número de ejemplos para poder particionar: 5, 10, 15
    'arbol__min_samples_split': range(2, 20)
}
kfold = StratifiedKFold(n_splits=10,random_state=357823,shuffle=True)
tuberia_arbol=Pipeline([('normalizador',normalizador),('arbol',DecisionTreeClassifier(random_state=357823))])
búsqueda_en_rejilla = GridSearchCV(tuberia_arbol,
                                   rejilla_de_hiperparámetros,
                                   scoring='accuracy',
                                   cv=kfold
                                   )
búsqueda_en_rejilla.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=357823, shuffle=True),
             estimator=Pipeline(steps=[('normalizador',
                                        ColumnTransformer(transformers=[('normalizador',
                                                                         MinMaxScaler(),
                                                                         [0, 1,
                                                                          2, 3,
                                                                          4,
                                                                          5])])),
                                       ('arbol',
                                        DecisionTreeClassifier(random_state=357823))]),
             param_grid={'arbol__max_depth': range(3, 11),
                         'arbol__min_samples_split': range(2, 20)},
             scoring='accuracy')

En este caso, los mejores hiperparámetros obtenidos son una profundidad máxima de 9 y un mínimo de 9 ejemplos para poder realizar una división en el árbol.

In [15]:
búsqueda_en_rejilla.best_params_

{'arbol__max_depth': 9, 'arbol__min_samples_split': 9}

Tasa de aciertos obtenido por este modelo en el proceso de validación cruzada:

In [16]:
print(búsqueda_en_rejilla.best_score_)

0.7262500000000001


Evaluamos el modelo sobre el conjunto de prueba (las filas 0,1 y 2 representan las distintas clases).

In [17]:

from sklearn.metrics import accuracy_score,classification_report
modelo_arbol=búsqueda_en_rejilla.best_estimator_
pred=modelo_arbol.predict(atributos_prueba)
print(accuracy_score(objetivo_prueba,modelo_arbol.predict(atributos_prueba)))
print(classification_report(objetivo_prueba,modelo_arbol.predict(atributos_prueba)))


0.717948717948718
              precision    recall  f1-score   support

           0       0.67      0.77      0.71        13
           1       0.81      0.77      0.79        22
           2       0.33      0.25      0.29         4

    accuracy                           0.72        39
   macro avg       0.60      0.60      0.60        39
weighted avg       0.71      0.72      0.71        39



Evaluamos el modelo sobre el conjunto de entrenamiento.

In [18]:
print(classification_report(objetivo_entrenamiento,modelo_arbol.predict(atributos_entrenamiento)))

              precision    recall  f1-score   support

           0       0.90      0.88      0.89        51
           1       0.91      0.97      0.94        86
           2       0.75      0.56      0.64        16

    accuracy                           0.90       153
   macro avg       0.85      0.80      0.82       153
weighted avg       0.89      0.90      0.89       153



In [19]:
from sklearn.neighbors import KNeighborsClassifier
rejilla_de_hiperparámetros = {
    # Máxima profundidad del árbol: 3, 4, 5, 6, 7, 8, 9, 10
    'knn__metric': ["euclidean","manhattan"],
    # Mínimo número de ejemplos para poder particionar: 5, 10, 15
    'knn__n_neighbors': range(1, 11)
}
kfold = StratifiedKFold(n_splits=10,random_state=357823,shuffle=True)
tuberia_arbol=Pipeline([('normalizador',normalizador),('knn',KNeighborsClassifier())])
búsqueda_en_rejilla = GridSearchCV(tuberia_arbol,
                                   rejilla_de_hiperparámetros,
                                   scoring='accuracy',
                                   cv=kfold)
búsqueda_en_rejilla.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=357823, shuffle=True),
             estimator=Pipeline(steps=[('normalizador',
                                        ColumnTransformer(transformers=[('normalizador',
                                                                         MinMaxScaler(),
                                                                         [0, 1,
                                                                          2, 3,
                                                                          4,
                                                                          5])])),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={'knn__metric': ['euclidean', 'manhattan'],
                         'knn__n_neighbors': range(1, 11)},
             scoring='accuracy')

In [20]:
búsqueda_en_rejilla.best_score_

np.float64(0.6545833333333333)

In [21]:
modelo_knn=búsqueda_en_rejilla.best_estimator_
print(accuracy_score(objetivo_prueba,modelo_knn.predict(atributos_prueba)))
print(classification_report(objetivo_prueba,modelo_knn.predict(atributos_prueba)))

0.5897435897435898
              precision    recall  f1-score   support

           0       0.50      0.54      0.52        13
           1       0.78      0.64      0.70        22
           2       0.29      0.50      0.36         4

    accuracy                           0.59        39
   macro avg       0.52      0.56      0.53        39
weighted avg       0.63      0.59      0.61        39



In [22]:
print(classification_report(objetivo_entrenamiento,modelo_knn.predict(atributos_entrenamiento)))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99        51
           1       1.00      0.99      0.99        86
           2       0.89      1.00      0.94        16

    accuracy                           0.99       153
   macro avg       0.96      0.99      0.98       153
weighted avg       0.99      0.99      0.99       153



In [23]:
from sklearn.preprocessing import OneHotEncoder
from keras.layers import Normalization,Dense
from keras import Sequential, Input
from sklearn.utils.class_weight import compute_class_weight
from keras.optimizers import Adam
import tensorflow as tf
import os
tf.random.set_seed(48679)
os.environ['PYTHONHASHSEED'] = '48679'
os.environ['TF_DETERMINISTIC_OPS'] = '1'

Construimos la red neuronal con una capa de entrada de 6 variables. Añadimos como capas ocultas una de 50 neuronas con función de activación ReLu y otra de 10 neuronas con función de activación tangente hiperbólica. Para la variable objetivo se ha utilizado una codificación one-hot y se ha utilizado la función softmax como función de activación. Además, se ha escogido como función de pérdida la entropía cruzada categórica y como optimizador Adam, que ha resultado dar un rendimiento bastante bueno.

In [ ]:
enc=OneHotEncoder()
objetivo_entrenamiento_cod=enc.fit_transform(objetivo_entrenamiento.to_frame()).toarray()
red = Sequential()
normalizador_red=Normalization()
normalizador_red.adapt(atributos_entrenamiento.to_numpy())
red.add(Input(shape=(6,)))
red.add(normalizador_red)
red.add(Dense(50,activation="relu"))
red.add(Dense(10,activation="tanh"))
red.add(Dense(3,activation="softmax"))
red.compile(optimizer=Adam(learning_rate=0.05), loss='categorical_crossentropy',
                   metrics=['accuracy'])
red.fit(atributos_entrenamiento,objetivo_entrenamiento_cod,batch_size=256,epochs=300)

[1.         0.59302326 3.1875    ]
Epoch 1/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.2157 - loss: 1.1819
Epoch 2/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.5621 - loss: 1.0034
Epoch 3/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.5621 - loss: 0.9554
Epoch 4/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.5229 - loss: 0.9423
Epoch 5/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.5033 - loss: 0.9337
Epoch 6/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.5686 - loss: 0.8949
Epoch 7/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.5686 - loss: 0.8733
Epoch 8/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.5621 - loss: 0.8771
Epoch 9/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.5621 - loss: 0.8826
Epoch 10/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.5752 - loss: 0.8758
Epoch 11/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.6013 - loss: 0.8634
Epoch 12/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

Evaluamos el moodelo sobre el conjunto de prueba.

In [25]:
pred=np.argmax(red.predict(atributos_prueba),axis=1)
print(classification_report(objetivo_prueba,pred))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
              precision    recall  f1-score   support

           0       0.64      0.69      0.67        13
           1       0.82      0.82      0.82        22
           2       0.33      0.25      0.29         4

    accuracy                           0.72        39
   macro avg       0.60      0.59      0.59        39
weighted avg       0.71      0.72      0.71        39



Evaluamos el modelo sobre el conjunto de entrenamiento.

In [26]:
pred_ent=np.argmax(red.predict(atributos_entrenamiento),axis=1)
print(classification_report(objetivo_entrenamiento,pred_ent))

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
              precision    recall  f1-score   support

           0       0.82      0.88      0.85        51
           1       0.92      0.90      0.91        86
           2       0.93      0.81      0.87        16

    accuracy                           0.88       153
   macro avg       0.89      0.86      0.87       153
weighted avg       0.89      0.88      0.88       153



Discretizamos los valores de las distintas propiedades, ya que Naive Bayes no trabaja con atributos continuos

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
discretizador = ColumnTransformer([('discretizador',
                                    KBinsDiscretizer(
    n_bins=5, 
    encode='ordinal',  
    strategy='uniform'  
),
                                   list(range(0,6)))])

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import CategoricalNB
rejilla_de_hiperparámetros = {
    'naive_bayes__alpha': range(1, 11)
    
}
kfold = StratifiedKFold(n_splits=5)
tuberia_arbol=Pipeline([('discretizador',discretizador),('naive_bayes',CategoricalNB())])
búsqueda_en_rejilla = GridSearchCV(tuberia_arbol,
                                   rejilla_de_hiperparámetros,
                                   scoring='accuracy',
                                   cv=kfold)
búsqueda_en_rejilla.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('discretizador',
                                        ColumnTransformer(transformers=[('discretizador',
                                                                         KBinsDiscretizer(encode='ordinal',
                                                                                          n_bins=1000,
                                                                                          strategy='uniform'),
                                                                         [0, 1,
                                                                          2, 3,
                                                                          4,
                                                                          5])])),
                                       ('naive_bayes', CategoricalNB())]),
             param_grid={'naive_bayes__alpha': range(1, 11)},
             scoring='accuracy')

Rendimiento obtenido en el proceso de validación cruzada.

In [60]:
búsqueda_en_rejilla.best_score_

np.float64(0.5559139784946237)

Evaluamos el modelo sobre el conjunto de prueba.

In [61]:
modelo_arbol=búsqueda_en_rejilla.best_estimator_
pred=modelo_arbol.predict(atributos_prueba)
print(classification_report(objetivo_prueba,modelo_arbol.predict(atributos_prueba)))

              precision    recall  f1-score   support

           0       1.00      0.08      0.14        13
           1       0.58      1.00      0.73        22
           2       0.00      0.00      0.00         4

    accuracy                           0.59        39
   macro avg       0.53      0.36      0.29        39
weighted avg       0.66      0.59      0.46        39



c:\Users\alber\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

Evaluamos el modelo sobre el conjunto de entrenamiento.

In [57]:
print(classification_report(objetivo_entrenamiento,modelo_arbol.predict(atributos_entrenamiento)))

              precision    recall  f1-score   support

           0       0.91      0.39      0.55        51
           1       0.66      1.00      0.79        86
           2       0.00      0.00      0.00        16

    accuracy                           0.69       153
   macro avg       0.52      0.46      0.45       153
weighted avg       0.67      0.69      0.63       153



c:\Users\alber\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo